In [15]:
def cleanToken(t):
    t = t.replace("...", "…")
    t = t.replace("tout.à.l'heure", "tout_à_l'heure")



    t = t.replace("<u>", "")
    t = t.replace("</u>", "")


    return t

In [16]:
import re

def alignTokLemmgloss(text, lemgloss):
    pattern = r'\[\[(\w+)\|\|(\w+)\]\]'
    lemgloss = re.sub(pattern, r'[[\1|\2]]', lemgloss)

    text_list = text.split("||")
    lemgloss_list = lemgloss.split("||")
    couple_list = []

    if any("KLT" in tok or "Sujet" in tok for tok in text_list) or any("KLT" in lem or "Sujet" in lem for lem in lemgloss_list):
        return couple_list

    text_list_stripped = [tok.strip() for tok in text_list]
    lemgloss_list_stripped = [lem.strip() for lem in lemgloss_list]

    if "standard" in text.lower() or "standard" in lemgloss.lower() or "Équivalent" in text or "Graphie" in text:
        return couple_list

    if text_list_stripped and text_list_stripped[-1] == "":
        text_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "<elles>":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "_":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and re.search(r'\d{4}', lemgloss_list_stripped[-1]):
        return couple_list
    if text_list_stripped and re.search(r'\d{4}', text_list_stripped[-1]):
        return couple_list
    if text_list_stripped and "colspan" in text_list_stripped[-1]:
        return couple_list
    if "[[*]]" in text_list_stripped:
        return couple_list

    if len(text_list_stripped) == len(lemgloss_list_stripped):
        for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
            tok = cleanToken(tok)
            lemmgloss = cleanToken(lemmgloss)
            if tok and lemmgloss:
                couple_list.append((tok, lemmgloss))
    else:
        punctuation_marks = ['!', '.', ',', '?', ';', ':']
        for punctuation in punctuation_marks:
            if text_list_stripped and text_list_stripped[-1].endswith(punctuation) and not lemgloss_list_stripped[-1].endswith(punctuation):
                lemgloss_list_stripped.append(punctuation)

        if len(text_list_stripped) == len(lemgloss_list_stripped):
            for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
                tok = cleanToken(tok)
                lemmgloss = cleanToken(lemmgloss)
                if tok and lemmgloss:
                    couple_list.append((tok, lemmgloss))
        else:
            for punctuation in punctuation_marks:
                if lemgloss_list_stripped and lemgloss_list_stripped[-1].endswith(punctuation) and not \
                text_list_stripped[-1].endswith(punctuation):
                    text_list_stripped.append(punctuation)

            if len(text_list_stripped) == len(lemgloss_list_stripped):
                for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
                    tok = cleanToken(tok)
                    lemmgloss = cleanToken(lemmgloss)
                    if tok and lemmgloss:
                        couple_list.append((tok, lemmgloss))
            else:
                with open("not_couple.txt", "a") as file:
                    file.write("text\n")
                    file.write(f"{text}\n")
                    file.write("lemgloss\n")
                    file.write(f"{lemgloss}\n")
    return couple_list

In [17]:
def whole_lines():
    filename = "all_line_objects.txt"
    all_examples = []

    with open(filename, "r") as file:
        lines = file.readlines()

    example = {}
    store_text = False
    store_lemgloss = False
    store_trans = False

    for line in lines:
        line = line.strip()

        if store_text:
            if line.startswith("||"):
                example["text"] = line[0:]
            else:
                example["text"] = line[3:]
            store_text = False
        elif store_lemgloss:
            example["lemgloss"] = line
            store_lemgloss = False
        elif store_trans:
            example['translation'] = line
            store_trans = False

            if example["text"] != "" and example["lemgloss"] != "None":
                all_examples.append(example)

            example = {}
        elif line == "text":
            store_text = True
        elif line == "lemgloss":
            store_lemgloss = True
        elif line == "translation":
            store_trans=True

    # Process each example
    for example in all_examples:
        text = example["text"]
        lemgloss = example["lemgloss"]
        trans = example['translation']


        example_info = alignTokLemmgloss(text, lemgloss)
        if example_info:
            with open('couple.txt','a') as t:
                t.write('text\n')
                t.write(f'{text}\n')
                t.write('translation\n')
                t.write(f'{trans}\n')

            with open('all_info.txt','a') as nfile:
                nfile.write('text\n')
                nfile.write(f'{text}\n')
                nfile.write('lemgloss\n')
                nfile.write(f'{lemgloss}\n')




whole_lines()

In [9]:
def couple2tokobjects(couple):
    text, lem = couple

    lem = re.sub(r'<sup>(\[\[[^\[\]]+\]\])(.*?)</sup>',
                 lambda m: ''.join(f'<sup>{x}</sup>' for x in re.findall(r'\[\[[^\[\]]+\]\]', m.group(0))), lem)
    lem = re.sub('particule o', 'o', lem)

    lem = re.sub(r'<font color=green>.*?</font color=green>', '', lem)

    if '[[R]]' not in lem:
        match = re.search(r'<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', lem)
        if match:
            k = match.group(2).strip(']')
            lem = re.sub(r'<sup>\[\[(?!(1|4)\]\]).*?</sup>', '[[R|_]]', lem)

        if "particule o" not in lem:
            lem = re.sub(r'<sup>\[\[4\]\]</sup>', '[[e|_]]', lem)

        lem = re.sub(r'<sup>\[\[1\]\]</sup>', '[[a|_]]', lem)

    if '[[R]]' in lem and '<sup>' in lem:
        match1 = re.search(r'\[\[R\]\]<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', lem)
        if match1:
            k = match1.group(2).strip(']')
            lem = re.sub(r'\[\[R\]\]<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', '[[R|_]]', lem)

        lem = re.sub(r'\[\[R\]\]<sup>\[\[4\]\]</sup>', '[[e|_]]', lem)
        lem = re.sub(r'\[\[R\]\]<sup>\[\[1\]\]</sup>', '[[a|_]]', lem)

    if '[[R]]' in lem and '<sup>' not in lem:
        lem = re.sub(r'\[\[R\]\]', '[[R|_]]', lem)
        k = 'R'

    if "'''" in text and '-' in lem and re.search(r'""".*?"""', text):
        extra_lem = re.search(r"'''(.*?)'''", text).group(1)
        match = re.search(r'(?<=\|{2}).*?-(.*?)(?=\|\|)', lem)

        extra_gloss = match.group(1).strip()
        lem = re.sub(r'-(.*?)\|\|', f'[[{extra_lem}|{extra_gloss}]] ||', lem)

    text_list = text.split('||')
    lem_list = lem.split('||')

    tok_objects = []

    for i in range(len(text_list)):
        tok = text_list[i].strip()
        lem_parts = lem_list[i].strip().split(']]')
        lem_sp = lem_list[i].strip().split('|')
        tok_list = tok.split(' ')

        if len(lem_sp) == 1 and tok.startswith("'''"):
            tok_objects.append({
                'tok': tok.replace("'''", ''),
                'lemma': tok.replace("'''", ''),
                'gloss': lem_parts[0]
            })


        elif len(lem_sp) == 1 and "<sub>" in lem_sp[0] and ('SC' or 'VP' or 'PredP' or 'CP' or 'DP') in lem_sp[0]:
            match = re.search(r'<sub>\[\[(.*?)\]\]', lem_sp[0])
            if match:
                content = match.group(1)
                tok_objects.append({
                    'PhraseStructure': content
                })

        elif len(lem_sp) == 1 and "<sub>" in lem_sp[0] and '[[' not in lem_sp[0]:
            match = re.search(r'<sub>(.*?)</sub>', lem_sp[0])
            if match:
                content = match.group(1)
                tok_objects.append({
                    'index': content
                })

        elif len(lem_sp) == 2 and '[[' in lem_sp[0]:

            gloss = lem_sp[1].rstrip('].')
            tok_objects.append({
                'tok': tok,
                'lemma': lem_sp[0].strip('[]'),
                'gloss': gloss
            })





        elif len(tok_list) == len(lem_parts) - 1:

            for i in range(len(tok_list)):
                lem_inside = lem_parts[i].split('|')[0]
                gloss_inside = lem_parts[i].split('|')[-1]

                tok_objects.append({'tok': tok_list[i],
                                    'lemma': lem_inside[3:],
                                    'gloss': gloss_inside

                                    })



        else:
            tok_objects.append({
                'tok': tok,
                'lemma': '_',
                'gloss': '_',
                'multiple': len(lem_parts) - 1})
            tok_obj = {'tok': tok, 'lemma': '_', 'gloss': '_', 'multiple': len(lem_parts) - 1}
            tok_n = 1

            for j in range(len(lem_parts)):
                if '[[' in lem_parts[j]:
                    lemma_gloss = lem_parts[j].strip('[[').split('|')
                    lemma = lemma_gloss[0].split('[[')[-1].strip()

                    if re.search(r'^-\w+', lemma):
                        if lemma != '-ig':
                            tok_objects.append({
                                'tok': lemma.strip(),
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 and tok != '-ig' else ''
                            })

                        else:
                            tok_objects.append({
                                'tok': lemma.strip(),
                                'lemma': lemma.strip(),
                                'gloss': 'petit'
                            })


                    elif lemma_gloss == ['R', '_']:
                        if k != '+C':
                            tok_objects.append({
                                'tok': k,
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                            })

                        else:
                            tok_objects.append({
                                'tok': k,
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else '',
                                'Epenthesis': 'Yes'
                            })

                    elif lemma_gloss == ['a', '_']:
                        tok_objects.append({
                            'tok': '1',
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                        })

                    elif lemma_gloss == ['e', '_']:
                        tok_objects.append({
                            'tok': '4',
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                        })

                    else:
                        tok_objects.append({
                            'tok': lemma.strip(),
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    if '-ig' in tok_obj['tok'] and '-ig' in tok_obj['lemma']:
                        tok_obj['gloss'] = 'petit'

                    tok_n += 1

    return tok_objects

In [10]:
relinks = re.compile(r"\[\[(.*?)\]\]")

def get_wikicode(title):
    wikicode=''
    # on traite les redirections vers d'autres pages
    if '#REDIRECTION' in pages.get(title, ''):             
        #on suit la redirection
        newtitle = relinks.search(pages[title]).group(1)                
        newtitle = newtitle[0].upper()+newtitle[1:]
        if '_' in newtitle: 
            newtitle = newtitle.replace("_", " ")            
        #et on regarde à nouveau si le titre est dans le dictionnaire pages
        if newtitle in pages:
            wikicode = pages[newtitle]              
        else: 
            newtitle = newtitle[0].upper()+newtitle[1:].replace(' ','_')
            if newtitle in pages:
                wikicode = pages[newtitle]
            else:
                newtitle = newtitle.split(',')[0]
                newtitle = newtitle[0].upper()+newtitle[1:]
                if newtitle in pages:
                    wikicode = pages[newtitle]
                else:
                    wikicode='__currentPage:\nstrange redirect to page that does not exist: '+newtitle
                    print(wikicode+'\n')

    else:
        wikicode = pages.get(title, '')  
    return wikicode

In [11]:
import re
import pickle

with open('Pages.pickle', 'rb') as f:
    pages = pickle.load(f)
open('pageTitles.txt','w', encoding="utf-8").write('\n'.join(pages.keys()))
print('extracted',len(pages),'pages')

extracted 9163 pages


In [12]:
rePOS = re.compile(r'articles?|art|verbes?|auxiliaires?|copules?|adverbes?|complémenteurs?|conjonctions?|prépositions?|adjectifs?|noms?|particules verbales?|interjections?|postpositions?|déterminants?|quantifieurs?|pronoms?|noms? propres?|suffixe|interrogatifs?|préfixes?|modaux|pluriels?|indéfinis?|particules? de discours|finales?|exclamatifs?', re.IGNORECASE)

lefffl = [li.strip() for li in open('lefff-2.1.txt').read().split('\n') if li.strip() and li[0]!='#']
lefff = {li.split('\t')[0]:li.split('\t')[2] for li in lefffl}
# print(lefff)


def add_pos(tok_obj):
    if 'pos' in tok_obj and tok_obj['pos'] : return tok_obj # pos already there

    # is the pos in the lemma?
    if 'lemma' in tok_obj:
        m = rePOS.search(tok_obj['lemma'])
        if m:
            tok_obj['pos'] = m.group(0)
            return tok_obj

    
    # is the pos in the gloss?
    if 'gloss' in tok_obj:
        m = rePOS.search(tok_obj['gloss'])
        if m:
            tok_obj['pos'] = m.group(0)
            return tok_obj

    
    # let's check the page for the lemma
    wikicode = get_wikicode(str.capitalize(tok_obj.get('lemma',''))) # TODO: improve as this might fail sometimes
    # print(wikicode[:222])
    m = rePOS.search(wikicode)
    if m:
        tok_obj['pos'] = m.group(0)
        return tok_obj
    
    # let's use the gloss to make an educated guess about the pos using lefff
    if tok_obj.get('gloss','') in lefff:
        tok_obj['pos'] = lefff[tok_obj.get('gloss','')]
    
    return tok_obj

    


In [13]:
import ast
import re


def generate_conll_output(tok_objects, text, text_fr, sent_id):
    conll_output = ""
    conll_id = 1
    inside_id = 1

    for tok_obj in tok_objects:
        if isinstance(tok_obj, dict) and len(tok_obj) == 1:
            field_key, field_value = next(iter(tok_obj.items()))
            conll_line = f"{conll_id}\t{field_key}\t{field_value}\t_\t_\t_\t_\t_\t_\t_\n"
        else:
            if 'multiple' not in tok_obj:
                
                conll_line = f"{conll_id}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t{tok_obj['pos']}\t_\t_\t_\t_\t_\tgloss={tok_obj['gloss']}\n"
                conll_id += 1
                inside_id = conll_id
            else:
                conll_line = f"{inside_id}-{inside_id + tok_obj['multiple'] - 1}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t_\t_\t_\t_\t_\t_\t_\n"

            conll_line = re.sub(']]', ' ', conll_line)
            conll_line = re.sub('\[\[kaout', 'kaout', conll_line)
            conll_line = re.sub("'''", '', conll_line)
            conll_line = re.sub('\[','',conll_line)

        conll_output += conll_line.strip() + "\n"

    conll_output = f"# sent_id = None__{sent_id}\n" + \
                   f"# text = {text.replace('||', '')}\n" + \
                   f"# text_fr = {text_fr}\n" + \
                   f"# status = WIP\n" + \
                   conll_output + "\n"

    return conll_output

In [15]:
import re
def process_tokens_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    conll_output = ""
    sent_id = 1  # Initialize sent_id to 1

    with open("couple.txt", "r") as couple_file:
        couple_lines = couple_file.readlines()
        couple_lines = [line.strip() for line in couple_lines]

    modified_tok_objects = []  # List to store modified token objects

    with open("couple_n.txt", "w") as output_file:  # Open the output file for writing
        for i, line in enumerate(lines):
            tok_objects = ast.literal_eval(line)
            modified_tok_objects.extend([add_pos(tok_obj) for tok_obj in tok_objects])

            text = couple_lines[i * 4 + 1].replace("text", "").strip()
            text = re.sub("'''", '', text)
            text_fr = couple_lines[i * 4 + 3].replace("translation", "").strip()

            # Update the sent_id value for each example
            conll_output += generate_conll_output


        conll_output += generate_conll_output(tok_objects, text, text_fr,sent_id)
        sent_id += 1  # Increment sent_id for the next example

    return conll_output


output = process_tokens_file('tokens.txt')
print(output)

KeyError: 'pos'